# Develop a LangGraph Agent

- 任意の LangGraph エージェントを AgentEngine でラップする
- https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/develop/langgraph#langgraph


In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]

'/home/mori/.gcloud/agent-engine-test-462504-001b7ad7b0a9.json'

## Vertex AI の初期化


In [3]:
import vertexai

vertexai.init(
    project="agent-engine-test-462504",
    location="asia-northeast1",  # Agent Engine にデプロイするときに使用する Bucket
    staging_bucket="gs://agent-engine-test-250610",
)

## LangGraph エージェントを構成する

- シンプルな build 済みの create_react_agent を利用
- langgraph_builder で囲む
- https://langchain-ai.github.io/langgraph/agents/agents/


In [4]:
from langgraph.prebuilt import create_react_agent


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"


def langgraph_builder(model, *, tools, **kwargs):
    # LanggraphAgentで渡されるmodelやtoolsを利用してReActエージェントを作成
    return create_react_agent(
        model=model,
        tools=tools,
    )

## ローカルでエージェントを起動する


In [18]:
from vertexai import agent_engines

# # Geminiを使う場合
# agent = agent_engines.LanggraphAgent(
#     model="gemini-1.5-flash-002",  # usでしか使えない
#     tools=[get_weather],
#     runnable_builder=langgraph_builder,
# )

In [19]:
# OpenAIを使う場合
from langchain_openai import ChatOpenAI


def model_builder(*, model_name, model_kwargs=None, **kwargs):
    return ChatOpenAI(model=model_name, **model_kwargs)


agent = agent_engines.LanggraphAgent(
    model="gpt-4.1-mini",
    model_builder=model_builder,
    tools=[get_weather],
    runnable_builder=langgraph_builder,
    model_kwargs={
        "api_key": os.environ["OPENAI_API_KEY"],
        "temperature": 0,
    },
)

In [20]:
# シンプルなクエリ
response = agent.query(
    input={"messages": [("user", "やあ")]},
)
response

{'messages': [{'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'schema', 'messages', 'HumanMessage'],
   'kwargs': {'content': 'やあ',
    'type': 'human',
    'id': '36340464-00dd-4121-831c-e6c248d8089a'}},
  {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'schema', 'messages', 'AIMessage'],
   'kwargs': {'content': 'やあ！こんにちは。今日はどんなことを話しましょうか？',
    'additional_kwargs': {'refusal': None},
    'response_metadata': {'token_usage': {'completion_tokens': 16,
      'prompt_tokens': 46,
      'total_tokens': 62,
      'completion_tokens_details': {'accepted_prediction_tokens': 0,
       'audio_tokens': 0,
       'reasoning_tokens': 0,
       'rejected_prediction_tokens': 0},
      'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
     'model_name': 'gpt-4.1-mini-2025-04-14',
     'system_fingerprint': 'fp_658b958c37',
     'id': 'chatcmpl-BglA39RVxeAVIN1dVAeY2P4cjagdm',
     'service_tier': 'default',
     'finish_reason': 'stop',
     'logprobs': None},


In [21]:
# ツール利用
response = agent.query(
    input={"messages": [("user", "今日の東京の天気は？")]},
)
response

{'messages': [{'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'schema', 'messages', 'HumanMessage'],
   'kwargs': {'content': '今日の東京の天気は？',
    'type': 'human',
    'id': '8e295ae4-af58-4274-adb0-7a0cee61a9cb'}},
  {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'schema', 'messages', 'AIMessage'],
   'kwargs': {'content': '',
    'additional_kwargs': {'tool_calls': [{'id': 'call_TTVZ0SVCEvc0TUoJ87z7TSC6',
       'function': {'arguments': '{"city":"東京"}', 'name': 'get_weather'},
       'type': 'function'}],
     'refusal': None},
    'response_metadata': {'token_usage': {'completion_tokens': 14,
      'prompt_tokens': 52,
      'total_tokens': 66,
      'completion_tokens_details': {'accepted_prediction_tokens': 0,
       'audio_tokens': 0,
       'reasoning_tokens': 0,
       'rejected_prediction_tokens': 0},
      'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
     'model_name': 'gpt-4.1-mini-2025-04-14',
     'system_fingerprint': 'fp_658b958

In [22]:
# イベントのストリーム表示
for chunk in agent.stream_query(
    input={"messages": [("user", "今日の東京の天気は？")]},
):
    print(chunk)
    print("=" * 100)

{'agent': {'messages': [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'tool_calls': [{'id': 'call_JWaumETWGpvYVtYW6U0ONfBd', 'function': {'arguments': '{"city":"東京"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, 'response_metadata': {'token_usage': {'completion_tokens': 14, 'prompt_tokens': 52, 'total_tokens': 66, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_658b958c37', 'id': 'chatcmpl-BglAI285lOHdXkoxkcoSWKQ00PCMS', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, 'type': 'ai', 'id': 'run--63301246-ec37-48a5-a1a5-3ac1589af736-0', 'tool_calls': [{'name': 'get_weather', 'args': {'city': '東京'}, 'id': 'call_JWaumETWGpvYVtYW6U0O

In [23]:
# トークンのストリーム表示
for chunk in agent.stream_query(
    input={"messages": [("user", "今日の東京の天気は？")]},
    stream_mode=["updates", "messages"],
):
    print(chunk)
    print("=" * 100)

['messages', [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessageChunk'], 'kwargs': {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_k8s4NkJSNnAf8iJeQkTYvZsY', 'function': {'arguments': '', 'name': 'get_weather'}, 'type': 'function'}]}, 'type': 'AIMessageChunk', 'id': 'run--728afc70-a5fe-4c9e-9ad3-8b257d7a47af', 'tool_calls': [{'name': 'get_weather', 'args': {}, 'id': 'call_k8s4NkJSNnAf8iJeQkTYvZsY', 'type': 'tool_call'}], 'tool_call_chunks': [{'name': 'get_weather', 'args': '', 'id': 'call_k8s4NkJSNnAf8iJeQkTYvZsY', 'index': 0, 'type': 'tool_call_chunk'}], 'invalid_tool_calls': []}}, {'langgraph_step': 1, 'langgraph_node': 'agent', 'langgraph_triggers': ['branch:to:agent'], 'langgraph_path': ['__pregel_pull', 'agent'], 'langgraph_checkpoint_ns': 'agent:0274f55f-6df1-2116-773e-221329863a3a', 'checkpoint_ns': 'agent:0274f55f-6df1-2116-773e-221329863a3a', 'ls_provider': 'openai', 'ls_model_name': 'gpt-4.1-mini', 'ls_model_typ

## エージェントのデプロイ

- IAM ロール `ストレージ管理者` を追加する


In [24]:
from vertexai.preview.reasoning_engines import ReasoningEngine

remote_agent = ReasoningEngine.create(
    agent,
    requirements=[
        "google-cloud-aiplatform[agent_engines,langchain]",
        "cloudpickle",
        "langgraph",
        "pydantic",
        "langchain-openai",
    ],
    display_name="Simple ReAct Agent",
    description="A simple ReAct agent with LangGraph",
    extra_packages=[],
)

Using bucket agent-engine-test-250610
Writing to gs://agent-engine-test-250610/reasoning_engine/reasoning_engine.pkl
Writing to gs://agent-engine-test-250610/reasoning_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://agent-engine-test-250610/reasoning_engine/dependencies.tar.gz
Creating ReasoningEngine
Create ReasoningEngine backing LRO: projects/983843513354/locations/asia-northeast1/reasoningEngines/8029249632531906560/operations/5959007226138984448
ReasoningEngine created. Resource name: projects/983843513354/locations/asia-northeast1/reasoningEngines/8029249632531906560
To use this ReasoningEngine in another session:
reasoning_engine = vertexai.preview.reasoning_engines.ReasoningEngine('projects/983843513354/locations/asia-northeast1/reasoningEngines/8029249632531906560')


# デプロイしたエージェントを確認する


In [25]:
from vertexai import agent_engines

for resource in agent_engines.list():
    print(resource)

resource name: projects/983843513354/locations/asia-northeast1/reasoningEngines/8029249632531906560


# デプロイしたエージェントを使用する


In [26]:
remote_agent = agent_engines.get(
    "projects/983843513354/locations/asia-northeast1/reasoningEngines/8029249632531906560"
)

In [27]:
# シンプルなクエリ
response = remote_agent.query(
    input={"messages": [("user", "やあ")]},
)
response

{'messages': [{'kwargs': {'content': 'やあ',
    'type': 'human',
    'id': '7a3ab65b-4013-4e33-bd2e-5d374a1a1d9f'},
   'type': 'constructor',
   'id': ['langchain', 'schema', 'messages', 'HumanMessage'],
   'lc': 1.0},
  {'kwargs': {'content': 'やあ！こんにちは。今日はどんなことを話しましょうか？',
    'tool_calls': [],
    'type': 'ai',
    'id': 'run--2c39556e-3a6c-42c2-9333-81e768385d22-0',
    'response_metadata': {'service_tier': 'default',
     'logprobs': None,
     'id': 'chatcmpl-BglNgH7AxyLIBUbs8HWbp0Ti9fRod',
     'model_name': 'gpt-4.1-mini-2025-04-14',
     'system_fingerprint': 'fp_658b958c37',
     'finish_reason': 'stop',
     'token_usage': {'prompt_tokens_details': {'audio_tokens': 0.0,
       'cached_tokens': 0.0},
      'total_tokens': 62.0,
      'prompt_tokens': 46.0,
      'completion_tokens': 16.0,
      'completion_tokens_details': {'rejected_prediction_tokens': 0.0,
       'reasoning_tokens': 0.0,
       'audio_tokens': 0.0,
       'accepted_prediction_tokens': 0.0}}},
    'additional_k

In [28]:
# ツール利用
response = remote_agent.query(
    input={"messages": [("user", "今日の東京の天気は？")]},
)
response

{'messages': [{'kwargs': {'content': '今日の東京の天気は？',
    'type': 'human',
    'id': 'b1400986-3da3-4a76-8eff-a9226e08546c'},
   'type': 'constructor',
   'id': ['langchain', 'schema', 'messages', 'HumanMessage'],
   'lc': 1.0},
  {'kwargs': {'content': '',
    'type': 'ai',
    'tool_calls': [{'type': 'tool_call',
      'args': {'city': '東京'},
      'id': 'call_7UAU6Q5RTMfvIO4UvoYNsOyc',
      'name': 'get_weather'}],
    'id': 'run--0e514800-4c84-418d-a51a-947e9d9689c2-0',
    'response_metadata': {'service_tier': 'default',
     'logprobs': None,
     'id': 'chatcmpl-BglNkMBFbvVwEHPIwOQzEEnRzh7OU',
     'model_name': 'gpt-4.1-mini-2025-04-14',
     'system_fingerprint': 'fp_658b958c37',
     'finish_reason': 'tool_calls',
     'token_usage': {'prompt_tokens_details': {'audio_tokens': 0.0,
       'cached_tokens': 0.0},
      'total_tokens': 66.0,
      'prompt_tokens': 52.0,
      'completion_tokens': 14.0,
      'completion_tokens_details': {'rejected_prediction_tokens': 0.0,
       'a

In [29]:
# トークンのストリーム表示
for chunk in remote_agent.stream_query(
    input={"messages": [("user", "今日の東京の天気は？")]},
    stream_mode=["updates", "messages"],
):
    print(chunk)
    print("=" * 100)

['messages', [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessageChunk'], 'kwargs': {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_oTVHFlPg7DUlnRVjwjbk5ICs', 'function': {'arguments': '', 'name': 'get_weather'}, 'type': 'function'}]}, 'type': 'AIMessageChunk', 'id': 'run--4dd8ca9f-82ed-49c7-a0eb-d2134df21632', 'tool_calls': [{'name': 'get_weather', 'args': {}, 'id': 'call_oTVHFlPg7DUlnRVjwjbk5ICs', 'type': 'tool_call'}], 'tool_call_chunks': [{'name': 'get_weather', 'args': '', 'id': 'call_oTVHFlPg7DUlnRVjwjbk5ICs', 'index': 0, 'type': 'tool_call_chunk'}], 'invalid_tool_calls': []}}, {'langgraph_step': 1, 'langgraph_node': 'agent', 'langgraph_triggers': ['branch:to:agent'], 'langgraph_path': ['__pregel_pull', 'agent'], 'langgraph_checkpoint_ns': 'agent:bbff072f-5838-c285-4be8-fa2855ae2a05', 'checkpoint_ns': 'agent:bbff072f-5838-c285-4be8-fa2855ae2a05', 'ls_provider': 'openai', 'ls_model_name': 'gpt-4.1-mini', 'ls_model_typ

## curl での呼び出し


In [30]:
%%bash
curl \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
https://asia-northeast1-aiplatform.googleapis.com/v1/projects/agent-engine-test-462504/locations/asia-northeast1/reasoningEngines/8029249632531906560

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

{
  "name": "projects/agent-engine-test-462504/locations/asia-northeast1/reasoningEngines/8029249632531906560",
  "displayName": "Simple ReAct Agent",
  "spec": {
    "packageSpec": {
      "pickleObjectGcsUri": "gs://agent-engine-test-250610/reasoning_engine/reasoning_engine.pkl",
      "requirementsGcsUri": "gs://agent-engine-test-250610/reasoning_engine/requirements.txt",
      "pythonVersion": "3.11"
    },
    "classMethods": [
      {
        "name": "query",
        "api_mode": "",
        "description": "Queries the Agent with the given input and config.\n\n        Args:\n            input (Union[str, Mapping[str, Any]]):\n                Required. The input to be passed to the Agent.\n            config (langchain_core.runnables.RunnableConfig):\n                Optional. The config (if any) to be used for invoking the Agent.\n            **kwargs:\n                Optional. Any additional keyword arguments to be passed to the\n                `.invoke()` method of the corresp

100 15434    0 15434    0     0  68437      0 --:--:-- --:--:-- --:--:-- 68292


In [31]:
%%bash
curl \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
https://asia-northeast1-aiplatform.googleapis.com/v1/projects/agent-engine-test-462504/locations/asia-northeast1/reasoningEngines/8029249632531906560:query -d '{
  "class_method": "query",
  "input": {
    "input": {
      "messages": [
        {
          "role": "user",
          "content": "今日の東京の天気は？"
        }
      ]
    }
  },
}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   197    0     0  100   197      0     37  0:00:05  0:00:05 --:--:--    37

{
  "output": {
    "messages": [
      {
        "lc": 1,
        "id": [
          "langchain",
          "schema",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "今日の東京の天気は？",
          "type": "human",
          "id": "d1d05d94-7877-458b-9c52-03ffc129eb5d"
        },
        "type": "constructor"
      },
      {
        "lc": 1,
        "kwargs": {
          "tool_calls": [
            {
              "name": "get_weather",
              "type": "tool_call",
              "id": "call_uRKsVb9UEr4JLa8H4VKvEz0p",
              "args": {
                "city": "東京"
              }
            }
          ],
          "invalid_tool_calls": [],
          "usage_metadata": {
            "output_tokens": 14,
            "output_token_details": {
              "audio": 0,
              "reasoning": 0
            },
            "input_tokens": 52,
            "total_tokens": 66,
            "input_token_details": {
              "cache_re

100  4934    0  4737  100   197    804     33  0:00:05  0:00:05 --:--:--  1292


etails": {
              "reasoning": 0,
              "audio": 0
            },
            "input_token_details": {
              "audio": 0,
              "cache_read": 0
            },
            "output_tokens": 12,
            "total_tokens": 92
          },
          "content": "今日の東京の天気は晴れです。",
          "type": "ai",
          "invalid_tool_calls": [],
          "additional_kwargs": {
            "refusal": null
          }
        },
        "type": "constructor",
        "id": [
          "langchain",
          "schema",
          "messages",
          "AIMessage"
        ]
      }
    ]
  }
}


## requests 経由での呼び出し


In [74]:
from google import auth as google_auth
from google.auth.transport import requests as google_requests
import requests
import json


def get_access_token():
    credentials, _ = google_auth.default(
        scopes=["https://www.googleapis.com/auth/cloud-platform"]
    )
    auth_request = google_requests.Request()
    credentials.refresh(auth_request)
    return credentials.token


token = get_access_token()

response = requests.post(
    "https://asia-northeast1-aiplatform.googleapis.com/v1/projects/agent-engine-test-462504/locations/asia-northeast1/reasoningEngines/8029249632531906560:streamQuery",
    headers={
        "Content-Type": "application/json",
        "Authorization": f"Bearer {token}",
    },
    data=json.dumps(
        {
            "class_method": "stream_query",
            "input": {
                "input": {
                    "messages": [
                        {
                            "role": "user",
                            "content": "今日の東京の天気は？",
                        }
                    ],
                }
            },
        }
    ),
    stream=True,
)

with response:
    for line in response.iter_lines():
        print(line.decode("utf-8"))
        print("=" * 100)

{"agent": {"messages": [{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "messages", "AIMessage"], "kwargs": {"content": "", "additional_kwargs": {"tool_calls": [{"id": "call_7lj7L07nk3fLvHkXodFgiCpz", "function": {"arguments": "{\"city\":\"\u6771\u4eac\"}", "name": "get_weather"}, "type": "function"}], "refusal": null}, "response_metadata": {"token_usage": {"completion_tokens": 14, "prompt_tokens": 52, "total_tokens": 66, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}, "model_name": "gpt-4.1-mini-2025-04-14", "system_fingerprint": "fp_658b958c37", "id": "chatcmpl-BgllaSa1Oc1VMWbbBDwrogasJZBVO", "service_tier": "default", "finish_reason": "tool_calls", "logprobs": null}, "type": "ai", "id": "run--336e9227-c702-4cf4-a793-5fbf2d8aea92-0", "tool_calls": [{"name": "get_weather", "args": {"city": "\u6771\u4eac"}, "id": "c